Las Losowy w wariancie Extra Trees.

In [1]:
# Libraries import
import numpy as np
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier
from sklearn.metrics import roc_auc_score, roc_curve, auc, precision_recall_curve, accuracy_score, recall_score, f1_score, make_scorer, confusion_matrix, log_loss
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
from sklearn.svm import SVC
import optuna
from sklearn.model_selection import cross_validate, GroupKFold
import pickle
from collections import Counter
import lightgbm as lgb
import scipy.stats
import os
from pathlib import Path

d:\GITHUB\REPOS\MACHINE-LEARNING-MODELS-RESEARCH\.CONDA\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


KeyboardInterrupt: 

This will be the main file of project.